In [ ]:
import os
import random
from glob import glob
from sklearn.metrics import f1_score, recall_score

import numpy as np
import xgboost as xgb

import settings as s

os.environ["EXSTRAQT_DATA_TYPE_FOLDER"] = s.OUTPUT_POSTFIX.lstrip("-")

In [ ]:
assert s.FILE_SIZE in ("Small", "Medium"), "Script suitable for `small` or `medium` datasets"

In [ ]:
SEED = int(os.environ.get("EXSTRAQT_SEED", 42))
print(f"{SEED=}")
random.seed(SEED)
np.random.seed(SEED)
EXSTRAQT_NUM_PROCS = int(os.environ.get("EXSTRAQT_NUM_PROCS", os.cpu_count()))

In [ ]:
location_main = os.path.join("features", os.environ["EXSTRAQT_DATA_TYPE_FOLDER"])

location_train_features_dm = f"{location_main}{os.sep}train_dm.bin"
location_valid_features_dm = f"{location_main}{os.sep}valid_dm.bin"
location_test_features_dm = f"{location_main}{os.sep}test_dm.bin"

In [ ]:
cuda_available = False
try:
    import torch
    cuda_available = torch.cuda.is_available()
except ImportError:
    pass

xgb_args = dict(
    seed=SEED,
    max_depth=6,
    scale_pos_weight=3,
    eta=0.3,
    subsample=1,
    colsample_bytree=0.5,
    num_parallel_tree=10,
    objective="binary:logistic",
    eval_metric="aucpr",
    disable_default_eval_metric=True,
    nthread=EXSTRAQT_NUM_PROCS,
    device="cpu",
)
if not s.HIGH_ILLICIT:
    if s.FILE_SIZE == "Small":
        xgb_args["eta"] = 0.1
    elif s.FILE_SIZE == "Medium":
        xgb_args["eta"] = 0.2
        xgb_args["scale_pos_weight"] = 4
    else:
        raise Exception("Use `training_large` for Large datasets!")
    xgb_args["max_depth"] = 5

if cuda_available:
    xgb_args["device"] = "cuda"
    xgb_args["nthread"] = 2

In [ ]:
%%time

validation_dm = xgb.DMatrix(location_valid_features_dm)

models = []
all_training_files = glob(f"{location_train_features_dm}{os.sep}*.bin") or [location_train_features_dm]
for index, fl in enumerate(sorted(all_training_files)):
    train_dm = xgb.DMatrix(fl)
    model = xgb.train(
        xgb_args,
        train_dm,
        num_boost_round=100,
        evals=[(validation_dm, "validation")],
        verbose_eval=True,
        early_stopping_rounds=10,
    )
    models.append(model)
    print(f"Trained {index + 1} of {len(all_training_files)} | {model.best_iteration=}")
    del train_dm
del validation_dm

In [ ]:
test_dm = xgb.DMatrix(location_test_features_dm)
y_test_predicted = []
for model in models:
    y_test_predicted.append(model.predict(test_dm, iteration_range=(0, model.best_iteration)))
y_test_predicted = np.max(y_test_predicted, axis=0) > 0.5
f1_test = f1_score(test_dm.get_label(), y_test_predicted) * 100
print(
    f"{SEED=}",
    f"f1={round(f1_test, 2)}",
    f"recall={round(recall_score(test_dm.get_label(), y_test_predicted) * 100, 2)}",
)
del test_dm
print(f1_test)